In [2]:
import pandas as pd
import re

In [ ]:
data = pd.read_excel(r'D:\WarsawParkInfo\data\table.xlsx', index_col='park_id')
data.head(7)

In [336]:
# Formating of area to digit format
area = data['area']
pattern = re.compile(r'\d+,*\d*')
digit = area.str.findall(pattern)
number = digit.apply(lambda lst: lst[0])

In [ ]:
# the table of Warsaw district.
html_tables = pd.read_html(r'https://pl.wikipedia.org/wiki/Podzia%C5%82_administracyjny_Warszawy')
districts = html_tables[0]['Dzielnica']
districts.index = pd.RangeIndex(len(districts), name='district_id')
districts = districts.str.replace('-', ' ')

In [ ]:
# districts in park table
pattern = re.compile('|'.join(districts))
p_districts = data['position']
p_districts = p_districts.str.findall(pattern).apply(lambda lst: lst[0] if len(lst) >0 else None)
p_districts

In [ ]:
# districts for the park table.
districts_index = pd.Index(districts)
district_id = districts_index.get_indexer(p_districts)

In [352]:
# streets address of parks. 
pattern = re.compile(r',',re.UNICODE)
buffer = data['position'].str.split(pattern, n=1, expand=True)
buffer = buffer.ffill(axis=1).iloc[:, 1]
position = buffer.apply(lambda s: s.strip())

In [368]:
# writing the data to df.
data['area'] = number
data['district_id'] = district_id
data['position'] = position

In [ ]:
# To write the data to excel
# with pd.ExcelWriter(r'D:\WarsawParkInfo\data\table.xlsx', mode='w') as writer:
#     data.to_excel(writer, sheet_name='Park', index=False)
#     districts.to_excel(writer, sheet_name='district', index=True)

In [4]:
# Creating of district of park table.
# key: Mokotow: 0, Ochota: 10, Śródmieście: 8.
data = pd.read_excel(r'D:\WarsawParkInfo\data\table.xlsx')
p_district_id = data[['district_id', 'park_id']]

# drop the record of park Pole-Mokotowskie
head, tail = p_district_id[:55], p_district_id[56:]
new_records = pd.DataFrame({'park_id': [55, 55, 55], 'district_id': [0, 10, 8]})
districts_of_park = pd.concat([head, tail, new_records])

In [ ]:
# To write the data to excel.
# with pd.ExcelWriter(r'D:\WarsawParkInfo\data\buffer.xlsx') as writer:
#     districts_of_park.to_excel(writer, sheet_name='Districts_of_park', index=False) 

In [9]:
# Coordinate of Warsaw districts. 
df = pd.read_csv(r'D:\WarsawParkInfo\data\district.csv', sep=';')
districts_coordinate = df[['WKT', 'name']]

In [46]:
def replace_pl_letters(record:pd.Series) -> pd.Series:
    transfer_map = str.maketrans({'Ą':'A','Ć':'C','Ę':'E','Ł':'L','Ń':'N','Ó':'O','Ś':'S','Ź':'Z','Ż':'Z','ą':'a','ć':'c','ę':'e','ł':'l','ń':'n','ó':'o','ś':'s','ź':'z','ż':'z'})
    return record.str.translate(transfer_map)
def format_url(park):
    park = re.sub(r'\W', '-', park)
    return re.sub(r'--+', '-', park)

In [403]:
# Coordinate of the Warsaw parks.
df = pd.read_csv(r'D:\WarsawParkInfo\data\park.csv', sep=';')
q_park: pd.DataFrame
q_park = df['website']
q_park.index = df['full_id']

e_park = pd.read_excel(r'D:\WarsawParkInfo\data\table.xlsx', sheet_name='Park')
e_park = e_park.iloc[:, [0, 1]]

pattern = re.compile('|'.join(e_park['park_name']), flags=re.IGNORECASE)

the_parks = q_park.str.findall(pattern)
the_parks = the_parks.explode()
# index maches
matches = the_parks[~the_parks.isnull()]
pattern = re.compile('|'.join(matches), flags=re.IGNORECASE)

matches.loc['w524003875'] = pd.NA
matches = matches.dropna()

In [420]:
is_fonded = df[df['full_id'].isin(matches.index)]
is_fonded.index = pd.Index(e_park['park_name'].str.lower()).get_indexer(matches)
# [123, 151, 36, 46, 233, 45, 51]
not_fonded = df.iloc[[123, 151, 36, 46, 233, 45, 51], :]
not_fonded.index =  pd.Index([4, 17, 26, 51, 63, 65, 67])

result = pd.concat([is_fonded, not_fonded])
result.index.name = 'park_id'
result = result.iloc[:, [0, -1]]

In [421]:
with pd.ExcelWriter(r'D:\WarsawParkInfo\data\buffer.xlsx') as writer:
    result.to_excel(writer, index=True)